In [1]:
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import UpdaterMD
from mindsponge import WithEnergyCell
from mindsponge.control import VelocityVerlet, Langevin, BerendsenBarostat
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.optimizer import SteepestDescent
from mindspore import context

In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])

Parameter (name=pbc_box, shape=(1, 3), dtype=Float32, requires_grad=True)

如果定义了周期性的盒子，那么还可以使用repeat_box功能来扩展/复制系统，比如可以这样将系统复制1000份

In [4]:
system.repeat_box([10, 10, 10])

Molecule<>

得到的分子系统的示意图如下所示：
![](../../docs/tutorial_b05.png)

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = SteepestDescent(system.trainable_params(), 1e-6)

In [7]:
sim = WithEnergyCell(system, potential, cutoff=1.0)
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Calculate the neighbour list using the inter-atomic distances.


[WARNING] KERNEL(331109,7fc8fc612500,python):2023-03-25-17:04:35.306.002 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel.cc:40] CheckDeviceSm] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6


In [8]:
run_info = RunInfo(10)
md.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-25 17:04:36
[MindSPONGE] Step: 0, E_pot: -8148.4155
[MindSPONGE] Step: 10, E_pot: -8682.079
[MindSPONGE] Step: 20, E_pot: -8715.706
[MindSPONGE] Step: 30, E_pot: -8718.024
[MindSPONGE] Step: 40, E_pot: -8718.347
[MindSPONGE] Step: 50, E_pot: -8718.359
[MindSPONGE] Step: 60, E_pot: -8718.654
[MindSPONGE] Step: 70, E_pot: -8718.68
[MindSPONGE] Step: 80, E_pot: -8718.667
[MindSPONGE] Step: 90, E_pot: -8718.668
[MindSPONGE] Finished simulation at 2023-03-25 17:04:42
[MindSPONGE] Simulation time: 6.48 seconds.
--------------------------------------------------------------------------------


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, temp),
                barostat=BerendsenBarostat(system, 1),
                velocity=velocity,
                time_step=5e-4)
md = Sponge(sim, optimizer=opt)

In [11]:
md.change_optimizer(opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b05.h5md', save_freq=10)

In [13]:
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-25 17:04:52
[MindSPONGE] Step: 0, E_pot: -8718.6875, E_kin: 10966.965, E_tot: 2248.2773, Temperature: 293.2139, Pressure: 145.61816, Volume: 64.0
[MindSPONGE] Step: 10, E_pot: -7376.2607, E_kin: 9809.651, E_tot: 2433.3906, Temperature: 262.27185, Pressure: -1758.2952, Volume: 63.981518
[MindSPONGE] Step: 20, E_pot: -8241.714, E_kin: 10733.006, E_tot: 2491.292, Temperature: 286.95874, Pressure: -148.62729, Volume: 63.963696
[MindSPONGE] Step: 30, E_pot: -7553.7617, E_kin: 10259.131, E_tot: 2705.3691, Temperature: 274.28915, Pressure: -2323.9917, Volume: 63.938736
[MindSPONGE] Step: 40, E_pot: -8418.33, E_kin: 11064.234, E_tot: 2645.9043, Temperature: 295.8145, Pressure: -935.2876, Volume: 63.911556
[MindSPONGE] Step: 50, E_pot: -8651.418, E_kin: 11306.675, E_tot: 2655.2568, Temperature: 302.29642, Pressure: -3316.3555, Volume: 63.877
[MindSPONGE] Step: 60, E_pot: -9305.027, E_kin: 11871.907, E_tot: 2566.88, Temperature: 317.40854, Pressure: -18